# CD2_P1

### Import Packages

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16


### Variable Paths, Execution Environments

In [2]:
modelname = 'CD2_P1'

I use 3 Local Machines & 2 Cloud Compute Engines (Google, Kaggle). The appropriate paths for each platform of execution are declared here.

In [3]:
Kaggle = 0
Colab = 0
Local = 1
LocalRM = 0
LocalOldLaptop = 0

# Kaggle Notebooks
Kaggle_Dataset = '/kaggle/input/catsdogsconv/KaggleCatsDogsConv'
Kaggle_SavedModels = '/kaggle/working/SavedModels'
Kaggle_TrainingHistory = '/kaggle/working/TrainingHistory'

# Google Drive
GD_Dataset = '/content/drive/MyDrive/Datasets/KaggleCatsDogs'
GD_SavedModels = '/content/drive/MyDrive/NotebookWorkspace/SavedModels'
CD_TrainingHistory = '/content/drive/MyDrive/NotebookWorkspace/TrainingHistory'

# Local Directories
Lc_Dataset = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\DatasetConv'
Lc_SavedModels = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\SavedModels'
Lc_TrainingHistory = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\Docs_Reports\\RawTrainingData'

Lc_RM_Dataset = "C:\\Users\\arisi\\Documents\\VSCode\\CatsDogs\\Dataset\\KaggleCatsDogsConv"
Lc_RM_SavedModels = "C:\\Users\\arisi\\Documents\\VSCode\\CatsDogs\\SavedModels"
Lc_RM_TrainingHistory = "C:\\Users\\arisi\\Documents\\VSCode\\CatsDogs\\Docs_Reports\\RawTrainingData"

Lc_Old_Dataset = 'Test'
Lc_Old_SavedModels = 'Test'
Lc_Old_TrainingHistory = 'Test'

In [4]:
if Kaggle:
    data_dir = Kaggle_Dataset
    SavedModelsPath = Kaggle_SavedModels
    TrainingHistoryPath = Kaggle_TrainingHistory
if Colab:
    from google.colab import drive
    drive.mount('/content/drive')
    data_dir = GD_Dataset
    SavedModelsPath = GD_SavedModels
    TrainingHistoryPath = CD_TrainingHistory
if Local:
    data_dir = Lc_Dataset
    SavedModelsPath = Lc_SavedModels
    TrainingHistoryPath = Lc_TrainingHistory
if LocalRM:
    data_dir = Lc_RM_Dataset
    SavedModelsPath = Lc_RM_SavedModels
    TrainingHistoryPath = Lc_RM_TrainingHistory
if LocalOldLaptop:
    data_dir = Lc_Old_Dataset
    SavedModelsPath = Lc_Old_SavedModels
    TrainingHistoryPath = Lc_Old_TrainingHistory

### Load the Dataset

In [5]:
from tensorflow.keras.utils import image_dataset_from_directory

In [6]:
# Load Datasets
train_dataset = image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    validation_split=0.2,  # 20% for validation
    subset='training',     # Use the 'training' subset
    seed=123
)

val_dataset = image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    validation_split=0.2,  # 20% for validation
    subset='validation',   # Use the 'validation' subset
    seed=123
)

Found 24997 files belonging to 2 classes.
Using 19998 files for training.
Found 24997 files belonging to 2 classes.
Using 4999 files for validation.


### Preprocessing & Augmentation

In [7]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [8]:
# Augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),  # 10% random rotation
    layers.RandomZoom(0.1),      # 10% zoom
    layers.RandomTranslation(0.1, 0.1),  # Random height and width shift
    layers.RandomBrightness(0.2)
])

# Augment the training data
def augment_img(image, label):
    image = data_augmentation(image)  # Apply augmentations
    return image, label

train_dataset = train_dataset.map(augment_img)

In [9]:
# Apply VGG-16 preprocessing
def preprocess_img(image, label):
    image = preprocess_input(image)  # Apply VGG16-specific preprocessing
    return image, label

train_dataset = train_dataset.map(preprocess_img)
val_dataset = val_dataset.map(preprocess_img)

### Load & Freeze the Pre-trained Model

In [10]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [11]:
# Freeze the layers of VGG16 so we don't retrain them
for layer in vgg_base.layers:
    layer.trainable = False

### Add the Head in the New Model & Compile

In [12]:
model = models.Sequential([
    vgg_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid') 
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Define the checkpoint path
name = modelname
checkpoint_path = f"{SavedModelsPath}\\CD2\\{name}_{{epoch:03d}}_val{{val_loss:.4f}}.keras"

from tensorflow.keras.callbacks import ModelCheckpoint

# Create the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',              # Save every epoch
    save_weights_only=False,
    save_best_only=False,           # Save every time, not just best
    monitor='val_loss',
    verbose=1,
)

### Train the Model

In [14]:
history = model.fit(
    train_dataset,
    epochs=3,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback]
)

Epoch 1/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9068 - loss: 0.3550
Epoch 1: saving model to C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\SavedModels\CD2_P1_001_val0.0532.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3093s 5s/step - accuracy: 0.9068 - loss: 0.3547 - val_accuracy: 0.9820 - val_loss: 0.0532
Epoch 2/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9602 - loss: 0.1017
Epoch 2: saving model to C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\SavedModels\CD2_P1_002_val0.0546.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3050s 5s/step - accuracy: 0.9602 - loss: 0.1017 - val_accuracy: 0.9820 - val_loss: 0.0546
Epoch 3/3
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9676 - loss: 0.0810
Epoch 3: saving model to C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\SavedModels\CD2_P1_003_val0.0486.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3623s 6s/step - accuracy: 0.9676 - lo

### Save History in JSON

In [15]:
import json

name = modelname
filepath = f"{TrainingHistoryPath}\\{name}.json"
with open(filepath, 'w') as f:
    json.dump(history.history, f)

### Load Chosen Model

In [10]:
filepath = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\SavedModels\\CD2\\CD2_P1_003_val0.0486.keras'
model = tf.keras.models.load_model(filepath)

### Continue Training

In [11]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
name = f'CD1_P1_continue'
checkpoint_path = f"{SavedModelsPath}\\CD2\\{name}_{{epoch:03d}}_val{{val_loss:.4f}}.keras"

from tensorflow.keras.callbacks import ModelCheckpoint

# Create the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',              # Save every epoch
    save_weights_only=False,
    save_best_only=False,           # Save every time, not just best
    monitor='val_loss',
    verbose=1,
)

from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

In [14]:
history2 = model.fit(
    train_dataset,
    epochs=4,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback, lr_scheduler]
)

Epoch 1/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9672 - loss: 0.0900
Epoch 1: saving model to C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\SavedModels\CD2\CD1_P1_continue_001_val0.0460.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 2977s 5s/step - accuracy: 0.9672 - loss: 0.0900 - val_accuracy: 0.9848 - val_loss: 0.0460 - learning_rate: 0.0010
Epoch 2/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9698 - loss: 0.0778
Epoch 2: saving model to C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\SavedModels\CD2\CD1_P1_continue_002_val0.0499.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 2972s 5s/step - accuracy: 0.9698 - loss: 0.0778 - val_accuracy: 0.9844 - val_loss: 0.0499 - learning_rate: 0.0010
Epoch 3/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9714 - loss: 0.0706
Epoch 3: saving model to C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\SavedModels\CD2\CD1_P1_continue

In [15]:
import json

name = 'CD1_P1_continue'
filepath = f"{TrainingHistoryPath}\\{name}.json"
with open(filepath, 'w') as f:
    json.dump(history2.history, f)

### Load Chosen Model

In [16]:
filepath = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\SavedModels\\CD2\\CD2_P1_003_val0.0486.keras'
model = tf.keras.models.load_model(filepath)

### Continue Training (B)

In [17]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
name = f'CD1_P1_continueB'
checkpoint_path = f"{SavedModelsPath}\\CD2\\{name}_{{epoch:03d}}_val{{val_loss:.4f}}.keras"

from tensorflow.keras.callbacks import ModelCheckpoint

# Create the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',              # Save every epoch
    save_weights_only=False,
    save_best_only=False,           # Save every time, not just best
    monitor='val_loss',
    verbose=1,
)

from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

In [19]:
history2b = model.fit(
    train_dataset,
    epochs=4,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback, lr_scheduler]
)

Epoch 1/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9721 - loss: 0.0738
Epoch 1: saving model to C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\SavedModels\CD2\CD1_P1_continueB_001_val0.0498.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 2969s 5s/step - accuracy: 0.9721 - loss: 0.0738 - val_accuracy: 0.9842 - val_loss: 0.0498 - learning_rate: 1.0000e-04
Epoch 2/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9747 - loss: 0.0676
Epoch 2: saving model to C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\SavedModels\CD2\CD1_P1_continueB_002_val0.0493.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 2977s 5s/step - accuracy: 0.9747 - loss: 0.0676 - val_accuracy: 0.9854 - val_loss: 0.0493 - learning_rate: 1.0000e-04
Epoch 3/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9726 - loss: 0.0708
Epoch 3: saving model to C:\Programming_Files\JupyterVSCode\Binary_Classification_Transfer_Learning\CatsDogs\SavedModels\CD2\CD1_P

In [20]:
import json

name = 'CD1_P1_continueB'
filepath = f"{TrainingHistoryPath}\\{name}.json"
with open(filepath, 'w') as f:
    json.dump(history2.history, f)